# Import des librairies

In [18]:
import os
import time
import warnings
from pathlib import Path
import json

import pandas as pd

!pip install azure-cognitiveservices-language-luis
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from azure.cognitiveservices.language.luis.authoring.models import (
    ApplicationCreateObject,
    ExampleLabelObject,
    EntityLabelObject,
)

!pip install msrest
from msrest.authentication import CognitiveServicesCredentials

from tqdm.notebook import tqdm_notebook as tqdm

!pip install python-dotenv
from dotenv import load_dotenv

# Chargement des fichiers

In [16]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

load_dotenv(override=True)

AZURE_LUIS_ENDPOINT_AUTHORING = os.getenv("AZURE_LUIS_ENDPOINT_AUTHORING")
AZURE_LUIS_ENDPOINT_PREDICTION = os.getenv("AZURE_LUIS_ENDPOINT_PREDICTION")
AZURE_LUIS_KEY_AUTHORING = os.getenv("AZURE_LUIS_KEY_AUTHORING")
AZURE_LUIS_KEY_PREDICTION = os.getenv("AZURE_LUIS_KEY_PREDICTION")


AZURE_LUIS_APP_NAME = "P10_LUIS"
AZURE_LUIS_APP_VERSION = "0.1"

AZURE_LUIS_APP_ID = "98264326-e211-4266-8056-612c47607ea1"

In [4]:
DATA_PATH = Path("/storage/P10/")
FRAMES_PATH = Path(DATA_PATH, "frames.json")

In [5]:
# instanciate LUIS Authoring Client
client = LUISAuthoringClient(
    AZURE_LUIS_ENDPOINT_AUTHORING, CognitiveServicesCredentials(AZURE_LUIS_KEY_AUTHORING)
)


In [6]:
## Create the LUIS Application

# define app basics
appDefinition = ApplicationCreateObject(
    name=AZURE_LUIS_APP_NAME,
    culture="en-us",
)

# create app
app_id = client.apps.add(appDefinition)

AZURE_LUIS_APP_ID = app_id

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID 98264326-e211-4266-8056-612c47607ea1


In [10]:
## Add Intents

book_id = client.model.add_intent(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="Book"
)
info_id = client.model.add_intent(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="Info"
)

In [11]:
## Add entities

# add Prebuilt entity
client.model.add_prebuilt(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    prebuilt_extractor_names=["geographyV2"],
)
client.model.add_prebuilt(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    prebuilt_extractor_names=["datetimeV2"],
)
client.model.add_prebuilt(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    prebuilt_extractor_names=["number"],
)

# add entity to app
or_city_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="or_city"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    entity_id=or_city_id,
    feature_relation_create_object={
        "model_name": "geographyV2",
    },
)
dst_city_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="dst_city"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    entity_id=dst_city_id,
    feature_relation_create_object={
        "model_name": "geographyV2",
    },
)
str_date_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="str_date"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    entity_id=str_date_id,
    feature_relation_create_object={
        "model_name": "datetimeV2",
    },
)
end_date_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="end_date"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    entity_id=end_date_id,
    feature_relation_create_object={
        "model_name": "datetimeV2",
    },
)
budget_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_APP_VERSION, name="budget"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_APP_VERSION,
    entity_id=budget_id,
    feature_relation_create_object={
        "model_name": "number",
    },
)

In [14]:
## Format data for LUIS

raw_data = pd.read_json(FRAMES_PATH)


entities = ["or_city", "dst_city", "str_date", "end_date", "budget"]
examples = []
unique_utterances = []

for turn in tqdm(raw_data["turns"]):
    for frame in turn:
        if frame["author"] == "wizard" or frame["text"] in unique_utterances:
            continue

        unique_utterances.append(frame["text"])

        is_book = False
        labels = []

        for act in frame["labels"]["acts_without_refs"]:
            for arg in act["args"]:
                if arg["key"] == "intent" and arg["val"] == "book":
                    is_book = True

                if (
                    arg["key"] in entities
                    and arg["val"] is not None
                    and frame["text"].find(arg["val"]) != -1
                ):
                    labels.append(
                        EntityLabelObject(
                            entity_name=arg["key"],
                            start_char_index=frame["text"].find(arg["val"]),
                            end_char_index=frame["text"].find(arg["val"])
                            + len(arg["val"]),
                        )
                    )

        if len(entities) > 0:
            examples.append(
                ExampleLabelObject(
                    text=frame["text"],
                    intent_name="Book" if is_book else "Info",
                    entity_labels=labels,
                )
            )

# add the examples in batch
batch_size = 100
for index in tqdm(range(0, len(examples), batch_size)):
    client.examples.batch(
        app_id=app_id,
        version_id=AZURE_LUIS_APP_VERSION,
        example_label_object_array=examples[index : index + batch_size],
    )

  0%|          | 0/1369 [00:00<?, ?it/s]

  0%|          | 0/97 [00:00<?, ?it/s]

In [6]:
client.train.train_version(AZURE_LUIS_APP_ID, AZURE_LUIS_APP_VERSION)
waiting = True
while waiting:
    info = client.train.get_status(AZURE_LUIS_APP_ID, AZURE_LUIS_APP_VERSION)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [7]:
# Mark the app as public so we can query it using any prediction endpoint.
# Note: For production scenarios, you should instead assign the app to your own LUIS prediction endpoint. See:
# https://docs.microsoft.com/en-gb/azure/cognitive-services/luis/luis-how-to-azure-subscription#assign-a-resource-to-an-app
client.apps.update_settings(AZURE_LUIS_APP_ID, is_public=True)

responseEndpointInfo = client.apps.publish(AZURE_LUIS_APP_ID, AZURE_LUIS_APP_VERSION, is_staging=False)

print(responseEndpointInfo)

{'additional_properties': {'directVersionPublish': False}, 'version_id': '0.1', 'is_staging': False, 'endpoint_url': 'https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/98264326-e211-4266-8056-612c47607ea1', 'region': 'westeurope, northeurope, uksouth, ukwest, francecentral, francesouth, norwayeast, norwaywest', 'assigned_endpoint_key': None, 'endpoint_region': 'westeurope', 'failed_regions': '', 'published_date_time': '2023-01-11T01:43:29Z'}


In [19]:
clientRuntime = LUISRuntimeClient(endpoint=AZURE_LUIS_ENDPOINT_PREDICTION, credentials=CognitiveServicesCredentials(AZURE_LUIS_KEY_PREDICTION))

# Production == slot name
predictionRequest = { "query" : "I want to book a ticket from Lyon to New York City with a budget of 500 euros maximum." }

predictionResponse = clientRuntime.prediction.get_slot_prediction(AZURE_LUIS_APP_ID, "Production", predictionRequest)
print("Top intent: {}".format(predictionResponse.prediction.top_intent))
print("Intents: ")

for intent in predictionResponse.prediction.intents:
    print("\t{}".format (json.dumps (intent)))
print("Entities: {}".format (predictionResponse.prediction.entities))

Top intent: Book
Sentiment: None
Intents: 
	"Book"
Entities: {'or_city': ['Lyon'], 'geographyV2': [{'value': 'Lyon', 'type': 'city'}, {'value': 'New York City', 'type': 'city'}], 'dst_city': ['New York City'], 'budget': ['500'], 'number': [500]}
